<a href="https://colab.research.google.com/github/Bytes-n-Bits/MIS2800/blob/main/HW3_2_AFC_North_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

print("Tools loaded and ready.")

Tools loaded and ready.


In [2]:
# Load the file
df = pd.read_csv('AFC_North_2025.csv')

# Clean up column names (SQL hates special characters like '-' or '%')
# This line renames "W-L%" to "win_loss_perc" so your queries are easier to write
df.columns = [c.replace(' ', '_').replace('-', '_').replace('%', 'perc') for c in df.columns]

print("CSV loaded into Python. Here is a preview:")
df.head()

CSV loaded into Python. Here is a preview:


,Tm,W,L,T,W_Lperc,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
0,New England Patriots*,14,3,0,0.824,490,320,170,10.0,-4.5,5.5,3.7,1.8
1,Buffalo Bills+,12,5,0,0.706,481,365,116,6.8,-2.3,4.5,4.9,-0.4
2,Miami Dolphins,7,10,0,0.412,347,424,-77,-4.5,-1.8,-6.3,-3.8,-2.5
3,New York Jets,3,14,0,0.176,300,503,-203,-11.9,-0.6,-12.5,-6.0,-6.5
4,Pittsburgh Steelers*,10,7,0,0.588,397,387,10,0.6,-0.5,0.1,-0.3,0.4


In [3]:
# 1. Create the database file (The 'House')
conn = sqlite3.connect('nfl_scouting.db')

# 2. Move the data into a table named 'afc_north_stats'
df.to_sql('afc_north_stats', conn, if_exists='replace', index=False)

print("SUCCESS: Data is now inside the SQL database!")

SUCCESS: Data is now inside the SQL database!


In [4]:
# The 'cursor' is our messenger
cursor = conn.cursor()

# Ask for the team names and wins
query = "SELECT Tm, W, L FROM afc_north_stats;"
results = cursor.execute(query).fetchall()

for row in results:
    print(row)

('New England Patriots*', 14, 3)
('Buffalo Bills+', 12, 5)
('Miami Dolphins', 7, 10)
('New York Jets', 3, 14)
('Pittsburgh Steelers*', 10, 7)
('Baltimore Ravens', 8, 9)
('Cincinnati Bengals', 6, 11)
('Cleveland Browns', 5, 12)
('Jacksonville Jaguars*', 13, 4)
('Houston Texans+', 12, 5)
('Indianapolis Colts', 8, 9)
('Tennessee Titans', 3, 14)
('Denver Broncos*', 14, 3)
('Los Angeles Chargers+', 11, 6)
('Kansas City Chiefs', 6, 11)
('Las Vegas Raiders', 3, 14)


# Filtering Data

In SQL, filtering is the process of narrowing down a dataset to retrieve only the specific rows that meet your requirements. This is done using the WHERE clause.

Row-by-Row Testing
When you execute a query with a WHERE clause, the database engine performs a logical "test" on every single row in the table.

* If a row passes the test (the condition is True), it is
included in the results.

* If a row fails the test (the condition is False), it is ignored.



The Three Components of a Filter
To build a filter, you typically need three pieces of information:

1. The Column: What category are you checking? (e.g., Wins)

2. The Operator: How are you comparing it? (e.g., =, >, <, LIKE)

3. The Value: What is the specific target? (e.g., 10)

*Example*: `SELECT * FROM nfl_stats WHERE Wins > 10`; In this scenario, SQL acts like a security guard at a gate, only allowing teams through if their "Wins" badge shows a number higher than 10.

In [5]:
# The Question: Show me teams with more than 8 wins
query = "SELECT Tm, W, L FROM afc_north_stats WHERE W > 8;"

# Execution
results = cursor.execute(query).fetchall()

print("TEAMS WITH WINNING RECORDS:")
for row in results:
    print(f"Team: {row[0]} | Wins: {row[1]}")

TEAMS WITH WINNING RECORDS:
Team: New England Patriots* | Wins: 14
Team: Buffalo Bills+ | Wins: 12
Team: Pittsburgh Steelers* | Wins: 10
Team: Jacksonville Jaguars* | Wins: 13
Team: Houston Texans+ | Wins: 12
Team: Denver Broncos* | Wins: 14
Team: Los Angeles Chargers+ | Wins: 11


In [6]:
# The Question: Which teams allowed more than 400 points against them?
query = "SELECT Tm, PA FROM afc_north_stats WHERE PA > 400;"

results = cursor.execute(query).fetchall()

print("TEAMS ALLOWING 400+ POINTS:")
for row in results:
    print(f"Team: {row[0]} | Points Against: {row[1]}")

TEAMS ALLOWING 400+ POINTS:
Team: Miami Dolphins | Points Against: 424
Team: New York Jets | Points Against: 503
Team: Cincinnati Bengals | Points Against: 492
Team: Indianapolis Colts | Points Against: 412
Team: Tennessee Titans | Points Against: 478
Team: Las Vegas Raiders | Points Against: 432


In [7]:
# The Question: High scorers who still lost a lot
query = "SELECT Tm, PF, L FROM afc_north_stats WHERE PF > 400 AND L > 8;"

results = cursor.execute(query).fetchall()

print("HIGH SCORERS WITH LOSING RECORDS:")
for row in results:
    print(f"Team: {row[0]} | Points For: {row[1]} | Losses: {row[2]}")

HIGH SCORERS WITH LOSING RECORDS:
Team: Baltimore Ravens | Points For: 424 | Losses: 9
Team: Cincinnati Bengals | Points For: 414 | Losses: 11
Team: Indianapolis Colts | Points For: 466 | Losses: 9


In [8]:
# The Question: Find any team that starts with the letter 'B'
# The % is a wildcard meaning "anything after the B"
query = "SELECT Tm FROM afc_north_stats WHERE Tm LIKE 'B%';"

results = cursor.execute(query).fetchall()

print("TEAMS STARTING WITH 'B':")
for row in results:
    print(row[0])

TEAMS STARTING WITH 'B':
Buffalo Bills+
Baltimore Ravens


# Sorting

In SQL, sorting is the process of organizing the rows in your result set into a specific order. This is handled by the ORDER BY clause.

Rearranging the Result Set
Unlike filtering (which removes data), sorting keeps all the rows but changes their sequence based on the values in one or more columns.

* ASC (Ascending): Sorts from smallest to largest (1 to 10) or alphabetically (A to Z). This is the default setting if you don't specify an order.

* DESC (Descending): Sorts from largest to smallest (10 to 1) or reverse-alphabetically (Z to A).

The Two Rules of Sorting
1. Placement Matters: The ORDER BY clause must always be the very last part of your query. If you put it before the WHERE clause, the database will return a syntax error.

2. Tie-Breakers: You can sort by multiple columns by separating them with a comma. SQL will sort by the first column first, and then use the second column only if there is a "tie" in the first one.

*Example*: `SELECT Tm, W FROM afc_north_stats ORDER BY W DESC`; In this scenario, SQL looks at the "Wins" column and puts the team with the most wins at the top of your list.

In [9]:
# Optional: Change the query string below to try different exercises
query = "SELECT Tm, W, L FROM afc_north_stats ORDER BY W DESC;"

# Send the messenger (cursor) to the database
results = conn.execute(query).fetchall()

# Print the results
print("--- ORGANIZED RESULTS ---")
for row in results:
    print(row)

--- ORGANIZED RESULTS ---
('New England Patriots*', 14, 3)
('Denver Broncos*', 14, 3)
('Jacksonville Jaguars*', 13, 4)
('Buffalo Bills+', 12, 5)
('Houston Texans+', 12, 5)
('Los Angeles Chargers+', 11, 6)
('Pittsburgh Steelers*', 10, 7)
('Baltimore Ravens', 8, 9)
('Indianapolis Colts', 8, 9)
('Miami Dolphins', 7, 10)
('Cincinnati Bengals', 6, 11)
('Kansas City Chiefs', 6, 11)
('Cleveland Browns', 5, 12)
('New York Jets', 3, 14)
('Tennessee Titans', 3, 14)
('Las Vegas Raiders', 3, 14)


In [10]:
# The Logic: Primary Sort = W (DESC), Secondary Sort = PD (DESC)
query = """
SELECT Tm, W, PD
FROM afc_north_stats
ORDER BY W DESC, PD DESC;
"""

results = cursor.execute(query).fetchall()

print("NFL STANDINGS (With PD Tie-Breaker):")
print("-" * 40)
for row in results:
    print(f"Team: {row[0]:<25} | Wins: {row[1]} | Point Diff: {row[2]}")

NFL STANDINGS (With PD Tie-Breaker):
----------------------------------------
Team: New England Patriots*     | Wins: 14 | Point Diff: 170
Team: Denver Broncos*           | Wins: 14 | Point Diff: 90
Team: Jacksonville Jaguars*     | Wins: 13 | Point Diff: 138
Team: Buffalo Bills+            | Wins: 12 | Point Diff: 116
Team: Houston Texans+           | Wins: 12 | Point Diff: 109
Team: Los Angeles Chargers+     | Wins: 11 | Point Diff: 28
Team: Pittsburgh Steelers*      | Wins: 10 | Point Diff: 10
Team: Indianapolis Colts        | Wins: 8 | Point Diff: 54
Team: Baltimore Ravens          | Wins: 8 | Point Diff: 26
Team: Miami Dolphins            | Wins: 7 | Point Diff: -77
Team: Kansas City Chiefs        | Wins: 6 | Point Diff: 34
Team: Cincinnati Bengals        | Wins: 6 | Point Diff: -78
Team: Cleveland Browns          | Wins: 5 | Point Diff: -100
Team: Las Vegas Raiders         | Wins: 3 | Point Diff: -191
Team: Tennessee Titans          | Wins: 3 | Point Diff: -194
Team: New York Jet